In [1]:
from metrics import metrics
from similarities.d2vk import D2VKSimilarity
metrics.train_test(metrics.get_metrics)('data/train.csv', 'data/test.csv', D2VKSimilarity(), True, 'D2VK')

Using TensorFlow backend.
Threshold = 0.7149, train accuracy = 0.9278


{'accuracy': 0.9524456521739131,
 'f1': 0.9517241379310344,
 'precision': 0.9663865546218487,
 'roc': 0.9904374409262761,
 'tp': 345,
 'tn': 356,
 'fp': 12,
 'fn': 23}

In [2]:
from metrics import x_feature_metrics

x_feature_metrics.lr_evaluate('data/train.csv', 'data/test.csv', D2VKSimilarity(), 'D2VK')

{'accuracy': 0.96875,
 'precision': 0.9434447300771208,
 'f1': 0.9696169088507265,
 'tp': 367,
 'tn': 346,
 'fp': 22,
 'fn': 1}

In [1]:
from helpers.similarity_generator import get_algorithm_by_name
from pipeline import ml
import pandas as pd

name = 'D2VK'
alg = get_algorithm_by_name(name, True)
model = ml.logistic_regression_train('data/train.csv', alg)
X, _ = ml.extract_features('data/unlabeled/junit4.csv', alg)

predictions = model.predict(X)
scores = model.predict_proba(X)
scores = scores[predictions == 1]
scores = scores[:, 1]

df = pd.read_csv('data/unlabeled/junit4.csv', index_col=0, na_filter=False)
df_up = df[predictions == 1]
df_up.drop(columns=['label', 'name1', 'name2'], inplace=True)
df_up.insert(4, 'score', scores)
df_up.sort_values(by='score', ascending=False, inplace=True)
df_up.to_csv('data/up/scores/' + name + '.csv')

print(name, len(df_up.index))

Using TensorFlow backend.
D2VK 464


In [2]:
import pickle
with open('data/up/accepted/all.pkl', 'rb') as f:
    all = pickle.load(f)
import pandas as pd
df = pd.read_csv('data/up/scores/D2VK.csv', index_col=0)
labels = [int(i in all) for i in df.index]
df.insert(5, 'label', labels)

names = ['LCS', 'COS', 'LEV', 'LSH', 'WMD', 'SiamX']
df_all = pd.concat([pd.read_csv('data/up/scores/'+name+'.csv', index_col=0) for name in names]).drop(columns=['score']).drop_duplicates()
ix_all = df_all.index.values
ix = df.index.values
import numpy as np
unique = np.setdiff1d(ix, ix_all)
df.loc[unique, 'label'] = -1

df.to_csv('data/up/D2VK_labeled.csv')
df.groupby('label').count()

,comment1,comment2,meta1,meta2,score
label,,,,,
-1,204,204,204,204,204
0,86,86,86,86,86
1,174,174,174,174,174
